## Import Packages and Data

## NOTE: Using HiGHS because Lauren is having an issue with Gurobi license

In [31]:
using LinearAlgebra
using Random
using DataFrames
using CSV
using Plots
using StatsBase
using Statistics
using JuMP
using HiGHS

In [33]:
# Paths (relative to notebook structure)
restaurant_path    = "../clean_data/restaurant_data.csv"
# scrap_path         = "../clean_data/food_scrap_locations.csv"
neighborhood_path  = "../clean_data/neighborhood_supply.csv"

# Read CSVs into DataFrames
restaurant_data       = CSV.read(restaurant_path, DataFrame);
# food_scrap_locations  = CSV.read(scrap_path, DataFrame)
neighborhood_supply   = CSV.read(neighborhood_path, DataFrame);

In [35]:
current_pantries       = CSV.read("../clean_data/Current Food Pantries.csv", DataFrame);

## Data Cleaning for Supply and Demand Data (Transshipment is already clean when just using current few)

In [38]:
# ================================
# CLEAN RESTAURANT DATA
# ================================
# Columns: latitude | longitude | waste
rename!(restaurant_data, names(restaurant_data)[3] => :supply)

# Ensure Float64
restaurant_data.supply    = Float64.(restaurant_data.supply)
restaurant_data.latitude  = Float64.(restaurant_data.latitude)
restaurant_data.longitude = Float64.(restaurant_data.longitude)

# *** DROP RESTAURANTS WITH NEGATIVE SUPPLY ***
filter!(row -> row.supply >= 0, restaurant_data)

# ================================
# CLEAN NEIGHBORHOOD SUPPLY DATA
# ================================
# Rename demand column for clarity
rename!(neighborhood_supply, names(neighborhood_supply)[4] => :supply_gap)

# Now neighborhood_supply.supply_gap might be String OR Float64.
# Only do replace/parse if it's strings.
if eltype(neighborhood_supply.supply_gap) <: AbstractString
    neighborhood_supply.supply_gap =
        parse.(Float64, replace.(neighborhood_supply.supply_gap, "," => ""))
end

# Demand = positive deficit, surplus -> 0
neighborhood_supply.demand = max.(0.0, -neighborhood_supply.supply_gap)

neighborhood_supply.latitude  = Float64.(neighborhood_supply.latitude)
neighborhood_supply.longitude = Float64.(neighborhood_supply.longitude)

# keep only necessary columns from neighborhood supply
select!(neighborhood_supply, [:latitude, :longitude, :demand, :Year])

# ================================
# SHOW CLEANED HEADS
# ================================
println("=== Restaurants (cleaned) ===")
println(first(restaurant_data, 5))

println("\n=== Neighborhood Supply (cleaned) ===")
println(first(neighborhood_supply, 5))

=== Restaurants (cleaned) ===
5×3 DataFrame
 Row │ latitude  longitude  supply  
     │ Float64   Float64    Float64 
─────┼──────────────────────────────
   1 │  40.6313   -73.9472  22593.5
   2 │  40.7144   -73.8319  54589.8
   3 │  40.7893   -73.9753  57175.8
   4 │  40.7498   -73.9728  26668.3
   5 │  40.7578   -73.9825  74368.1

=== Neighborhood Supply (cleaned) ===
5×4 DataFrame
 Row │ latitude  longitude  demand     Year  
     │ Float64   Float64    Float64    Int64 
─────┼───────────────────────────────────────
   1 │  40.8267   -73.9217  1.02143e5   2025
   2 │  40.8321   -73.8887  3.33493e5   2025
   3 │  40.8016   -73.9374  0.0         2025
   4 │  40.8469   -73.8918  1.13653e5   2025
   5 │  40.6193   -73.9733  0.0         2025


# Fixed-Center Stochastic Redistribution Model (Formulation 3-F)

We formulate the expected-cost, fixed-center redistribution problem below.  
The set of open donation centers is assumed to be fixed in advance.

---

**Sets**

- $R$: restaurants (supply nodes)  
- $\bar D$: *fixed* set of open donation centers (transshipment nodes)  
- $N$: neighborhoods (demand nodes)  
- $S$: demand scenarios  

---

**Parameters**

- $a_i$: available surplus at restaurant $i \in R$  
- $d_k^s$: demand at neighborhood $k \in N$ in scenario $s \in S$  
- $p_s$: probability of scenario $s \in S$  
- $c_{ij}$: cost of transporting one unit from restaurant $i$ to donation center $j$  
- $c_{jk}$: cost of transporting one unit from donation center $j$ to neighborhood $k$  
- $\overline{Q}_j$: capacity of donation center $j \in \bar D$  
- $M$: large penalty coefficient for unmet demand  
- $\lambda_{\text{dist}}$: weight on expected distance / transportation cost and unmet-demand penalties  
- $\lambda_{\text{eq}}$: weight on expected equity (maximum shortfall) term  

---

**Decision Variables (scenario-dependent)**

For each scenario $s \in S$:

- $x_{ij}^s \ge 0$: shipment from restaurant $i \in R$ to donation center $j \in \bar D$  
- $y_{jk}^s \ge 0$: shipment from donation center $j \in \bar D$ to neighborhood $k \in N$  
- $u_k^s \ge 0$: unmet demand at neighborhood $k \in N$  
- $r_k^s \ge 0$: food received by neighborhood $k \in N$  
- $t^s \ge 0$: maximum shortfall across neighborhoods in scenario $s$  

---

### **Objective: Minimize Expected Cost + Equity Penalty**

$ \displaystyle
\min\;
\lambda_{\text{dist}}
\sum_{s \in S} p_s \left(
    \sum_{i \in R} \sum_{j \in \bar D} c_{ij} x_{ij}^s
    +
    \sum_{j \in \bar D} \sum_{k \in N} c_{jk} y_{jk}^s
    +
    M \sum_{k \in N} u_k^s
\right)
\;+\;
\lambda_{\text{eq}} \sum_{s \in S} p_s t^s
$

---

### **Constraints**

**1. Restaurant supply limits (per scenario)**  

For all $i \in R$, $s \in S$:
$ \displaystyle
\sum_{j \in \bar D} x_{ij}^s \;\le\; a_i
$

---

**2. Flow conservation at fixed open donation centers**  

For all $j \in \bar D$, $s \in S$:
$ \displaystyle
\sum_{k \in N} y_{jk}^s
\;=\;
\sum_{i \in R} x_{ij}^s
$

---

**3. Neighborhood received food definition**  

For all $k \in N$, $s \in S$:
$ \displaystyle
r_k^s
\;=\;
\sum_{j \in \bar D} y_{jk}^s
$

---

**4. Cannot exceed neighborhood demand**  

For all $k \in N$, $s \in S$:
$ \displaystyle
r_k^s \;\le\; d_k^s
$

---

**5. Maximum shortfall representation**  

For all $k \in N$, $s \in S$:
$ \displaystyle
t^s \;\ge\; d_k^s - r_k^s
$

---

**6. Demand balance (received + unmet = total)**  

For all $k \in N$, $s \in S$:
$ \displaystyle
\sum_{j \in \bar D} y_{jk}^s \;+\; u_k^s \;=\; d_k^s
$

---

**7. Capacity limits at fixed open centers**  

For all $j \in \bar D$, $s \in S$:
$ \displaystyle
\sum_{i \in R} x_{ij}^s \;\le\; \overline{Q}_j
$

---

**8. Nonnegativity**

For all valid indices $i, j, k, s$:
$ \displaystyle
x_{ij}^s \ge 0,\quad
y_{jk}^s \ge 0,\quad
r_k^s \ge 0,\quad
u_k^s \ge 0,\quad
t^s \ge 0
$


In [41]:
# extract vectors
R = nrow(restaurant_data)        # number of restaurants
D = nrow(current_pantries)       # number of (fixed) donation centers

# use one year (e.g., 2023) to define the set of neighborhoods
nbhd_2023 = neighborhood_supply[neighborhood_supply.Year .== 2023, :]
N = nrow(nbhd_2023)              # number of neighborhoods

S = 3                            # 3 demand scenarios (2023, 2024, 2025)

supply = restaurant_data.supply  # a_i

# --- Scenario-specific demands ---
demand_2023 = neighborhood_supply[neighborhood_supply.Year .== 2023, :].demand
demand_2024 = neighborhood_supply[neighborhood_supply.Year .== 2024, :].demand
demand_2025 = neighborhood_supply[neighborhood_supply.Year .== 2025, :].demand

# Make an N x S "demand matrix" (columns = scenarios)
demand_df = DataFrame(
    y2023 = demand_2023,
    y2024 = demand_2024,
    y2025 = demand_2025,
)

# If you still want an M "big penalty", you can set it from average demand:
average_demand = mean(Matrix(demand_df), dims = 2)
M = sum(average_demand)  # or maximum(average_demand), or just a big constant


5.081311007183482e7

In [43]:
# manhattan distance
manhattan(lat1, lon1, lat2, lon2) = abs(lat1 - lat2) + abs(lon1 - lon2)

# define cost matrices

# cij: Restaurants (i) → Donation centers (j)
cij = [
    manhattan(
        restaurant_data.latitude[i], restaurant_data.longitude[i],
        current_pantries.Latitude[j], current_pantries.Longitude[j]
    )
    for i in 1:R, j in 1:D
]

# cjk: Donation centers (j) → Neighborhoods (k)  (USE 2023 SUBSET!)
cjk = [
    manhattan(
        current_pantries.Latitude[j], current_pantries.Longitude[j],
        nbhd_2023.latitude[k], nbhd_2023.longitude[k]
    )
    for j in 1:D, k in 1:N
]

println("Size of cij (R x D): ", size(cij))
println("Size of cjk (D x N): ", size(cjk))


Size of cij (R x D): (319, 6)
Size of cjk (D x N): (6, 197)


In [45]:
function build_stochastic_combined_model(w_cost::Float64, w_eq::Float64;
                                         p_s::Vector{Float64}=fill(1.0 / S, S))
    @assert length(p_s) == S "Length of p_s must equal S"

    model = Model(HiGHS.Optimizer)
    set_silent(model)

    # ==========================
    # Decision variables
    # ==========================
    @variable(model, x[1:R, 1:D, 1:S] >= 0)   # restaurant -> center, by scenario
    @variable(model, y[1:D, 1:N, 1:S] >= 0)   # center -> neighborhood, by scenario
    @variable(model, u[1:N, 1:S] >= 0)        # unmet demand, by scenario
    @variable(model, r[1:N, 1:S] >= 0)        # received, by scenario
    @variable(model, t[1:S]      >= 0)        # worst unmet demand in each scenario

    # ==========================
    # Constraints
    # ==========================

    # 1) Restaurant supply (same supply in every scenario)
    @constraint(model, [i in 1:R, s in 1:S],
        sum(x[i, j, s] for j in 1:D) <= supply[i]
    )

    # 2) Flow conservation at centers (per scenario)
    @constraint(model, [j in 1:D, s in 1:S],
        sum(y[j, k, s] for k in 1:N) == sum(x[i, j, s] for i in 1:R)
    )

    # 3) Demand balance for cost part:
    #    inflow + unmet = demand^s
    @constraint(model, [k in 1:N, s in 1:S],
        sum(y[j, k, s] for j in 1:D) + u[k, s] == demand_df[k, s]
    )

    # 4) r_k^s = received = ∑_j y_jk^s
    @constraint(model, [k in 1:N, s in 1:S],
        r[k, s] == sum(y[j, k, s] for j in 1:D)
    )

    # 5) Cannot exceed scenario-specific demand: r_k^s ≤ d_k^s
    @constraint(model, [k in 1:N, s in 1:S],
        r[k, s] <= demand_df[k, s]
    )

    # 6) Worst unmet demand per scenario: t^s ≥ d_k^s − r_k^s
    @constraint(model, [k in 1:N, s in 1:S],
        t[s] >= demand_df[k, s] - r[k, s]
    )

    # (Optional) Capacity at centers (if you have Q̄_j defined as a vector of length D)
    # @constraint(model, [j in 1:D, s in 1:S],
    #     sum(x[i, j, s] for i in 1:R) <= Qbar[j]
    # )

    # ==========================
    # Objective components
    # ==========================

    # Expected cost:
    #   E_s[ ∑ c_ij x_ij^s + ∑ c_jk y_jk^s + M ∑ u_k^s ]
    @expression(model, cost_expr,
        sum(
            p_s[s] * (
                sum(cij[i, j] * x[i, j, s] for i in 1:R, j in 1:D) +
                sum(cjk[j, k] * y[j, k, s] for j in 1:D, k in 1:N) +
                M * sum(u[k, s] for k in 1:N)
            )
            for s in 1:S
        )
    )

    # Expected equity term: E_s[t^s]
    @expression(model, equity_expr,
        sum(p_s[s] * t[s] for s in 1:S)
    )

    # Weighted objective: w_cost * E[cost] + w_eq * E[equity]
    @objective(model, Min, w_cost * cost_expr + w_eq * equity_expr)

    return model, x, y, u, r, t, cost_expr, equity_expr
end

build_stochastic_combined_model (generic function with 1 method)

In [47]:
function compute_stats_stochastic(model, x, y, u, r, t;
                                  p_s::Vector{Float64}=fill(1.0 / S, S))

    @assert length(p_s) == S "Length of p_s must equal S"

    # Solve the model
    optimize!(model)

    # === Extract solution ===
    x_val = value.(x)   # size: (R, D, S)
    y_val = value.(y)   # size: (D, N, S)
    u_val = value.(u)   # size: (N, S)
    r_val = value.(r)   # size: (N, S)
    t_val = value.(t)   # size: (S,)

    # ==========================
    # Scenario-wise totals
    # ==========================

    # total flow R→D per scenario
    total_flow_rest_to_center_s = [sum(x_val[:, :, s]) for s in 1:S]

    # total flow D→N per scenario
    total_flow_center_to_neighborhood_s = [sum(y_val[:, :, s]) for s in 1:S]

    # total unmet per scenario
    total_unmet_s = [sum(u_val[:, s]) for s in 1:S]

    # worst unmet per scenario (t^s)
    worst_unmet_s = t_val[:]

    # ==========================
    # Expected totals (using p_s)
    # ==========================

    expected_total_flow_rest_to_center =
        sum(p_s[s] * total_flow_rest_to_center_s[s] for s in 1:S)

    expected_total_flow_center_to_neighborhood =
        sum(p_s[s] * total_flow_center_to_neighborhood_s[s] for s in 1:S)

    expected_total_unmet =
        sum(p_s[s] * total_unmet_s[s] for s in 1:S)

    expected_worst_unmet =
        sum(p_s[s] * worst_unmet_s[s] for s in 1:S)

    # ==========================
    # Distance / Cost metrics
    # ==========================

    # Distance per scenario
    total_distance_ij_s = [
        sum(cij[i, j] * x_val[i, j, s] for i in 1:R, j in 1:D)
        for s in 1:S
    ]

    total_distance_jk_s = [
        sum(cjk[j, k] * y_val[j, k, s] for j in 1:D, k in 1:N)
        for s in 1:S
    ]

    total_distance_s = [total_distance_ij_s[s] + total_distance_jk_s[s] for s in 1:S]

    # Expected distances
    expected_total_distance_ij =
        sum(p_s[s] * total_distance_ij_s[s] for s in 1:S)
    expected_total_distance_jk =
        sum(p_s[s] * total_distance_jk_s[s] for s in 1:S)
    expected_total_distance =
        sum(p_s[s] * total_distance_s[s] for s in 1:S)

    # ==========================
    # Per-neighborhood expected stats
    # ==========================

    # expected demand per neighborhood (from demand_df)
    expected_demand_k = [
        sum(p_s[s] * demand_df[k, s] for s in 1:S)
        for k in 1:N
    ]

    # expected received per neighborhood
    expected_received_k = [
        sum(p_s[s] * r_val[k, s] for s in 1:S)
        for k in 1:N
    ]

    # expected unmet per neighborhood
    expected_unmet_k = [
        sum(p_s[s] * u_val[k, s] for s in 1:S)
        for k in 1:N
    ]

    demand_vs_received_expected = [
        (expected_demand_k[k], expected_received_k[k], expected_unmet_k[k])
        for k in 1:N
    ]

    # ==========================
    # Pack results
    # ==========================
    return (
        # scenario-wise totals
        total_flow_rest_to_center_s = total_flow_rest_to_center_s,
        total_flow_center_to_neighborhood_s = total_flow_center_to_neighborhood_s,
        total_unmet_s = total_unmet_s,
        worst_unmet_s = worst_unmet_s,
        total_distance_ij_s = total_distance_ij_s,
        total_distance_jk_s = total_distance_jk_s,
        total_distance_s = total_distance_s,

        # expectations
        expected_total_flow_rest_to_center = expected_total_flow_rest_to_center,
        expected_total_flow_center_to_neighborhood = expected_total_flow_center_to_neighborhood,
        expected_total_unmet = expected_total_unmet,
        expected_worst_unmet = expected_worst_unmet,
        expected_total_distance_ij = expected_total_distance_ij,
        expected_total_distance_jk = expected_total_distance_jk,
        expected_total_distance = expected_total_distance,

        # per-neighborhood expected stats
        expected_demand_k = expected_demand_k,
        expected_received_k = expected_received_k,
        expected_unmet_k = expected_unmet_k,
        demand_vs_received_expected = demand_vs_received_expected,

        # objective
        objective_value = objective_value(model)
    )
end


compute_stats_stochastic (generic function with 1 method)

In [49]:
years = [2023, 2024, 2025]

model, x, y, u, r, t, cost_expr, equity_expr =
    build_stochastic_combined_model(1.0, 0.7)

stats = compute_stats_stochastic(model, x, y, u, r, t)

results_stoch = DataFrame(
    Year = Int[],
    TotalDistance = Float64[],
    WorstUnmetDemand = Float64[]
)

for s in 1:S
    push!(results_stoch, (
        years[s],
        stats.total_distance_s[s],   # distance in scenario s
        stats.worst_unmet_s[s]       # t[s] in scenario s
    ))
end

results_stoch


Row,Year,TotalDistance,WorstUnmetDemand
,Int64,Float64,Float64
1,2023,2.61502e6,1.3806e6
2,2024,2.73337e6,1.90143e6
3,2025,2.56297e6,2.329e6
